In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


import string
import re
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import FreqDist
from nltk.tokenize import word_tokenize

# Import Data

In [ ]:
final_df = pd.read_csv('csv_files/cleaned_items_reviews_df.csv',index_col=0)
final_df.head()

In [ ]:
# drop rows where cleaned_reviews are na
final_df.isna()
final_df = final_df.dropna(subset=['cleaned_reviews'])

# Class Imbalance

In [ ]:
final_df['user_rating'].value_counts()

In [ ]:
# plt.figure(figsize=(20,10))
final_df['user_rating'].value_counts().plot.bar(rot=0)

plt.title('Count of User Rating',fontsize=20)
plt.xlabel('Rating',fontsize=15)
plt.xticks(fontsize=10)
plt.ylabel('Count',fontsize=15)
plt.yticks(fontsize=10);

It appears we have a class imbalance problem where the number of 5.0 reviews are much greater than the other reviews. Oversampling the minority classes will lead to overfitting due to a creation of large number of synthetic rows. To address this, I will take a set number of random samples from each class.

## Undersampling

In [ ]:
def undersampling(dataframe, column, list_user_rating, samples):
    df = pd.DataFrame()
    for rating in list_user_rating:
        sample = dataframe[dataframe[column] == rating].sample(samples)
        df = df.append(sample)
    return df

In [ ]:
undersampled_df =undersampling(final_df, 'user_rating', final_df['user_rating'].unique(), 250)
undersampled_df.shape

# TF-IDF Vectorization

Stands for Term Frequency, Inverse Document Frequency and weighs each word in a document by how unique it is. The idea behind TF-IDF is that the words that occur less in all the documents and more in individual documents are weighted more.

In [ ]:
# declare variables for reviews and rating
reviews = undersampled_df['cleaned_reviews'].values
rating = undersampled_df['user_rating'].values

In [ ]:
# convert text into TF-IDF feature vectors

vectorizer = TfidfVectorizer(max_features=2500, # num of most frequent occuring words to create bag of words vector
                             min_df=7, # minimum words that occur in at least 7 documents
                             max_df=0.8, # words that occur in a maximum of 80% of documents
                             stop_words=stopwords.words('english'))

processed_features = vectorizer.fit_transform(reviews).toarray()

In [ ]:
print(vectorizer)
print(processed_features)

# Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(processed_features,
                                                    rating, 
                                                    test_size=0.2, # 80% training/20% test
                                                    random_state=0)

# Random Forest

## GridSearchCV

In [ ]:
# # Perform a gridsearch for Random Forest
# # Obtain optimal values of model hyperparameters

# RF_classifier = RandomForestClassifier(random_state=0)

# params = {'criterion': ['gini','entropy'], # Measure the quality of a split
#              'max_depth': [10, 20, 30], # Maximum number of levels in tree
#              'max_features': ['auto', 'sqrt'], # Number of features to consider at every split
#              'min_samples_leaf': [1, 2, 4], # Minimum number of samples required at each leaf node
#              'min_samples_split': [2, 5, 10], # Minimum number of samples required to split a node
#              'n_estimators': [200, 500, 1000]} # The number of trees in the forest

# g_s_RF = GridSearchCV(RF_classifier,param_grid=params, n_jobs=-1) # create gridsearch with input params
# grid_result = g_s_RF.fit(X_train, y_train) # fit gridsearch with training data
# best_params = grid_result.best_params_ # output best params for given data
# best_params

In [ ]:
RF_classifier = RandomForestClassifier(criterion='gini',
                                       max_depth=10,
                                       max_features='auto',
                                       min_samples_leaf=4,
                                       min_samples_split=2,
                                       n_estimators=1000, 
                                       random_state=0)
RF_classifier.fit(X_train, y_train)

In [ ]:
RF_predictions = RF_classifier.predict(X_test)
RF_predictions

## Results

### Confusion Matrix

In [ ]:
print(confusion_matrix(y_test, RF_predictions))

### Classification Report

In [ ]:
print(classification_report(y_test, RF_predictions))

### Accuracy Score

In [ ]:
print(accuracy_score(y_test, RF_predictions))

In [ ]:
accuracy_results = pd.DataFrame({'Model':['Random Forest'], 'Accuracy':[accuracy_score(y_test, RF_predictions)]})
accuracy_results

### Plot Comparison between Actuals and Predicted

In [ ]:
predictions_count = dict(Counter(RF_predictions))
predictions_count_df = pd.DataFrame.from_dict(predictions_count, orient ='index', columns =['Predictions'])

y_test_count = dict(Counter(y_test))
y_test_count_df = pd.DataFrame.from_dict(y_test_count, orient ='index', columns =['Actuals'])

random_forest_df = pd.concat([predictions_count_df, y_test_count_df], axis=1).fillna(0)

In [ ]:
random_forest_df.plot.bar(rot=0, figsize=(20,10))

plt.title('Comparison between Actual Reviews and Predicted Reviews (Random Forest)',fontsize=30)

plt.xlabel('Rating',fontsize=25)
plt.xticks(fontsize=20)
plt.ylabel('Count',fontsize=25)
plt.yticks(fontsize=20)
plt.legend(fontsize=20);

# K-Nearest Neighbors (KNN)

In [ ]:
# # Perform a gridsearch for Random Forest
# # Obtain optimal values of model hyperparameters

# KNN_classifier = KNeighborsClassifier()

# params = {'n_neighbors': [5,15,25], # Number of neighbors
#              'weights': ['uniform','distance']} # Weight function used in prediction

# g_s_KNN = GridSearchCV(KNN_classifier,param_grid=params, n_jobs=-1) # create gridsearch with input params
# grid_result = g_s_KNN.fit(X_train, y_train) # fit gridsearch with training data
# best_params = grid_result.best_params_ # output best params for given data
# best_params

In [ ]:
KNN_classifier = KNeighborsClassifier(n_neighbors=15,weights='distance')
KNN_classifier.fit(X_train, y_train)

In [ ]:
KNN_predictions = KNN_classifier.predict(X_test)
KNN_predictions

## Results

### Confusion Matrix

In [ ]:
print(confusion_matrix(y_test, KNN_predictions))

### Classification Report

In [ ]:
print(classification_report(y_test, KNN_predictions))

### Accuracy Score

In [ ]:
print(accuracy_score(y_test, KNN_predictions))

In [ ]:
accuracy_results = accuracy_results.append({'Model':'KNN', 'Accuracy':accuracy_score(y_test, KNN_predictions)},ignore_index=True)
accuracy_results

### Plot Comparison between Actuals and Predicted

In [ ]:
predictions_count = dict(Counter(KNN_predictions))
predictions_count_df = pd.DataFrame.from_dict(predictions_count, orient ='index', columns =['Predictions'])

y_test_count = dict(Counter(y_test))
y_test_count_df = pd.DataFrame.from_dict(y_test_count, orient ='index', columns =['Actuals'])

KNN_df = pd.concat([predictions_count_df, y_test_count_df], axis=1).fillna(0)

In [ ]:
random_forest_df.plot.bar(rot=0, figsize=(20,10))

plt.title('Comparison between Actual Reviews and Predicted Reviews (KNN)',fontsize=30)

plt.xlabel('Rating',fontsize=25)
plt.xticks(fontsize=20)
plt.ylabel('Count',fontsize=25)
plt.yticks(fontsize=20)
plt.legend(fontsize=20);

# SVM

In [ ]:
SVC_classifier = SVC(kernel='linear')
SVC_classifier.fit(X_train, y_train)

In [ ]:
SVC_predictions = SVC_classifier.predict(X_test)
SVC_predictions

## Results

### Confusion Matrix

In [ ]:
print(confusion_matrix(y_test, SVC_predictions))

### Classification Report

In [ ]:
print(classification_report(y_test, SVC_predictions))

### Accuracy Score

In [ ]:
print(accuracy_score(y_test, SVC_predictions))

In [ ]:
accuracy_results = accuracy_results.append({'Model':'SVC', 'Accuracy':accuracy_score(y_test, SVC_predictions)},ignore_index=True)
accuracy_results

### Plot Comparison between Actuals and Predicted

In [ ]:
predictions_count = dict(Counter(SVC_predictions))
predictions_count_df = pd.DataFrame.from_dict(predictions_count, orient ='index', columns =['Predictions'])

y_test_count = dict(Counter(y_test))
y_test_count_df = pd.DataFrame.from_dict(y_test_count, orient ='index', columns =['Actuals'])

KNN_df = pd.concat([predictions_count_df, y_test_count_df], axis=1).fillna(0)

In [ ]:
random_forest_df.plot.bar(rot=0, figsize=(20,10))

plt.title('Comparison between Actual Reviews and Predicted Reviews (KNN)',fontsize=30)

plt.xlabel('Rating',fontsize=25)
plt.xticks(fontsize=20)
plt.ylabel('Count',fontsize=25)
plt.yticks(fontsize=20)
plt.legend(fontsize=20);

# Sentiment Analyzer using NLTK Module Vader

* works well with social media type text
* doesn't require any training data and is constructed using human-curated sentiment lexicon

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_analyzer_scores(sentence):
    # polarity_scores: ranging from -1 (most neg) to 1 (most pos)
    score = analyser.polarity_scores(sentence)
    return score # {neg: Negative, neu: Neutral, pos: Positive, compound: Aggregated Score}

In [ ]:
def compound_score(text):
    comp=sentiment_analyzer_scores(text)
    return comp['compound'] # returns the compound score from the dictionary

In [ ]:
# applying on the reviews column to get the score
final_df['sentiment_score'] = final_df['review_description'].apply(lambda x:compound_score(x))

In [ ]:
final_df.head()

In [ ]:
def sentiment_category(score):
    if score >= 0.6:
        return 5.0
    elif score >= 0.2:
        return 4.0
    elif score >= -0.2:
        return 3.0
    elif score >= -0.6:
        return 2.0
    else:
        return 1.0

In [ ]:
final_df['sentiment_score_rating'] = final_df['sentiment_score'].apply(lambda x: sentiment_category(x))

In [ ]:
final_df.head()

## Results

### Confusion Matrix

In [ ]:
cnf_matrix = confusion_matrix(final_df['user_rating'],final_df['sentiment_score_rating'])
cnf_matrix

### Classification Report

In [ ]:
print(classification_report(final_df['user_rating'],final_df['sentiment_score_rating']))

### Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score
vader_accuracy_score = accuracy_score(final_df['user_rating'],final_df['sentiment_score_rating'])
vader_accuracy_score

In [ ]:
accuracy_results = accuracy_results.append({'Model':'Vader', 'Accuracy':vader_accuracy_score},ignore_index=True)
accuracy_results

### Plot Comparison between Actuals and Predicted

In [ ]:
review_rating_counts = final_df['user_rating'].value_counts()
sentiment_score_rating_counts = final_df['sentiment_score_rating'].value_counts()

review_df = pd.concat([review_rating_counts, sentiment_score_rating_counts], axis=1)

review_df.plot.bar(rot=0, figsize=(20,10))

plt.title('Comparison between Actual Reviews and Predicted Reviews',fontsize=30)

plt.xlabel('Rating',fontsize=25)
plt.xticks(fontsize=20)
plt.ylabel('Count',fontsize=25)
plt.yticks(fontsize=20)
plt.legend(fontsize=20);

# Predictive Accuracy Results

In [ ]:
accuracy_results